In [3]:
import pandas as pd

# PostgreSql скрипт

``` PostgreSql
delete from deal_status a using deal_status b where a=b and a.ctid < b.ctid;

drop view if EXISTS w1 cascade;
create view w1 as 
SELECT DISTINCT id,
	timestamp,
    status,
    DENSE_RANK() over (partition by id order by timestamp) as rank,
    lead(timestamp) over (partition by id order by timestamp) as next_status
FROM deal_status
order by id, timestamp;

with w2 as(
select *, next_status - timestamp as diff, (next_status - timestamp)::FLOAT / 60 as diff_minuts
from w1
order by id, timestamp)

select status, ROUND(avg(diff_minuts)::numeric, 1) as time_in_minutes
from w2
group by status
having avg(diff_minuts) is not NULL
```

# Результат работы скрипта

In [9]:
dp = pd.read_csv("Output.csv")

In [10]:
dp

,status,time_in_minutes
0,Ожидает подтверждения,0.1
1,Проверка товара на складе,337.1
2,Товар не устраивает,1002.7
3,Формирование счета для юр.лица,0.0
4,Ожидание юр.лица,72.7
5,Сбор информации,6.8
6,Ожидание,378.6
7,Согласование нового товара,78.6
8,Заведение заказа в 1С,39.7
9,Проверка информации,32.2


# Немного о написании скрипта

Изначально методом внимательного взгляда и ранжировки были обнаружены дубликаты строк.
Они не несут никакой новой информации, поэтому первым делом избавляемся от них.

Далее необходимо было  в отсортированном наборе для каждого статуса поднять момент его изменения, если таков имеется.
Для этого отлично подойдут оконные функции, в частности LEAD() - возвращает следующее значение в отсортированном окне.

Остается дело за малым, используя AVG() для посчитанной разницы в минутах и группируя по всем статусам, получаем искомые результаты. Скрипт должен отрабатывать корректно, так как наша оконная функция возвращает NULL, если смены статуса не было, а агрегация не использует NULL значения.

# Дополнительный вопрос

Анализируя полученный результат, можно сказать что самым долгим является статус "Товар не устраивает". Так же выбиваются из общего настроения статусы "Ожидание" и "Проверка товара на складе". Но, как я могу судить, эти статусы зависимы от действий покупателя(кроме Проверки товара на складе, однако это ручной физический труд, чем и определяется долгий статус).

Но полученные результаты не отражают полной картины, так как мы не видим текущий статус. Если у заявки конечный статус("Сделка успешна"),  то проблем нет, однако мы не оцениваем висящие заявки в промежуточных статусах, следовательно анализ полученного результата не имеет большого смысла.

Для более честного анализа необходимо добавить подсчет висящих статусов от настоящего момента. Условно считая, что если статус не менялся, то мы все равно оцениваем его как NOW() - timestamp и используем в дальнейшей агрегации. Можно выкинуть из расчетов конечный статус, чтоб не нагружать систему, так как нам эта информация не интересна.

# Дополнительная задача

Не совсем понятно для какой выборки отображать текущий статус. Для полученной выборки это делать нельзя, так как у нас нет заявок, а идет агрегация по всем заявкам. А отображать текущую заявку в исходных данных нет смысла, мы можем вытащить его в любой момент.